In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from random import randint
from time import sleep 
import os
import pickle

In [2]:
brand_df = pd.read_csv('github_data_active.csv').drop(columns=['Unnamed: 0'])

# Drop inactive websites
brand_df.drop(brand_df.loc[brand_df['website_active']==False].index, inplace=True)

In [3]:
brand_df.head()

,brand,url,label,website_active
0,#ladc,https://www.ladc-paris.com,NOT,True
1,'47,https://47brand.com,NOT,True
2,- contraer -,https://contraer.com,NOT,True
3,032c,https://032c.com,SUS,True
5,11 degrees,https://11degrees.com,NOT,True


In [4]:
# Takes: brand (str)
# Returns: url for amazon brand (str)
def create_brand_url_amazon(brand):
    brand_formatted = brand.replace(' ', '+')
    amazon_brand_url = 'https://www.amazon.com/s?k=' + brand_formatted
    return amazon_brand_url

In [5]:
brand_df['amazon_url'] = brand_df['brand'].apply(create_brand_url_amazon)

In [6]:
brand_df.head()

,brand,url,label,website_active,amazon_url
0,#ladc,https://www.ladc-paris.com,NOT,True,https://www.amazon.com/s?k=#ladc
1,'47,https://47brand.com,NOT,True,https://www.amazon.com/s?k='47
2,- contraer -,https://contraer.com,NOT,True,https://www.amazon.com/s?k=-+contraer+-
3,032c,https://032c.com,SUS,True,https://www.amazon.com/s?k=032c
5,11 degrees,https://11degrees.com,NOT,True,https://www.amazon.com/s?k=11+degrees


In [7]:
# Takes: product soup (str), brand (mainly str)
# Returns: True (if brand is mentioned within product listing) or False
def find_product_relevant(product, brand):
    brand = str(brand)
    try:
        product_brand = product.find('span', attrs= {'class': 'a-size-base-plus a-color-base'}).get_text().lower()
    except AttributeError:
        return False
    
    if brand in product_brand:
        return True
    else:
        return False
    


In [8]:
# Takes: soup of the Amazon brand (str), i (int), brand (str)
# Returns: product listing url (str)
def grab_product_listing_url(soup, i, brand): 
    
    try:
        # finds indv. product
        product = soup.find('div', attrs= {'data-index': i})
        
        # determines if product is relevant to brand
        product_rel = find_product_relevant(product, brand)
        
        # if product is relevant, find asin
        if product_rel:
            asin = product.get('data-asin')
            product_listing_url = 'https://www.amazon.com/dp/' + asin
        # if product is not relevant, return empty str
        else:
            product_listing_url = ''
    
    # if attribute error, return empty str
    except AttributeError:
        product_listing_url = ''
        
    
    return product_listing_url
    

In [9]:
os.mkdir("amazon_asins")

FileExistsError: [Errno 17] File exists: 'amazon_asins'

In [11]:
# brand_list and product_url_list will be used to make DF
brand_list = []
product_url_list = []

# used for pickle
loop_brand_list = []

In [12]:
# user_agents_list = [
#     "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-GB; rv:1.8.1.6)",
#     "Gecko/20070725 Firefox/2.0.0.6",
#     "Mozilla/5.0 (X11; U; Linux i686; en-US) AppleWebKit/534.3 (KHTML, like Gecko) Chrome/6.0.472.63 Safari/534.3",
#     "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
#     "Opera/9.00 (Windows NT 5.1; U; en)",
#     "Mozilla/5.0 (Linux; U; Android 0.5; en-us) AppleWebKit/522+ (KHTML, like Gecko) Safari/419.3",
#     "Mozilla/5.0 (iPhone; U; CPU like Mac OS X; en) AppleWebKit/420+ (KHTML, like Gecko) Version/3.0 Mobile/1A543a Safari/419.3",
# ]

In [ ]:
# Iterate through each brand
for i,r in brand_df.iterrows():
    
    brand_url = r['amazon_url']
    brand = r['brand'] 
    
    loop_brand_list.append(brand)
    
    for brand_item in loop_brand_list:
        # save brand
        current_brands = os.listdir("amazon_asins")
        current_brands = [f.split('.pkl')[0] for f in current_brands]
        
        # if brand has not been searched, create soup 
        if brand_item not in current_brands:  
    
    
            # Create Soup for Brand 
            HEADERS = ({'User-Agent':
                            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
                            'Accept-Language': 'en-US, en;q=0.5'})
            r = requests.get(brand_url, headers=HEADERS)
            soup = BeautifulSoup(r.content, 'lxml')

            # Create 10 listings per brand
            product_count = 1 
            while product_count < 11:

                product_url = grab_product_listing_url(soup, product_count, brand)
                print(product_url)


                # Save information to lists
                brand_list.append(brand)
                product_url_list.append(product_url)


                product_count += 1


            sleep(randint(1,4))
            
            # save product_urls 
                with open("amazon_asins/{}.pkl".format(brand_item),'wb') as f:
                    pickle.dump(product_url,f)

In [19]:
product_df = pd.DataFrame(list(zip(brand_list, product_url_list)),
               columns =['brand', 'product_url'])
product_df

,brand,product_url
0,#ladc,
1,#ladc,
2,#ladc,
3,#ladc,
4,#ladc,
...,...,...
17275,zhrill,
17276,zhrill,
17277,zhrill,
17278,zhrill,


In [20]:
product_df.to_csv('product_url_df.csv')

In [5]:
with open("amazon_asins/airblaster.pkl","rb") as f:
    data = pickle.load(f)

In [6]:
data

'https://www.amazon.com/dp/B09JD92D85'